In [1]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from nltk import word_tokenize
from collections import Counter, defaultdict
import random

# Reuters dataset

In [2]:
first_sentence = reuters.sents()[0]
print(first_sentence)  # [u'ASIAN', u'EXPORTERS', u'FEAR', u'DAMAGE', u'FROM' ...

# Get the bigrams
print(list(bigrams(first_sentence)))  # [(u'ASIAN', u'EXPORTERS'), (u'EXPORTERS', u'FEAR'), (u'FEAR', u'DAMAGE'), (u'DAMAGE', u'FROM'), ...

# Get the padded bigrams
print(list(bigrams(first_sentence, pad_left=True, pad_right=True)))  # [(None, u'ASIAN'), (u'ASIAN', u'EXPORTERS'), (u'EXPORTERS', u'FEAR'), (u'FEAR', u'DAMAGE'), (u'DAMAGE', u'FROM'),

# Get the trigrams
print(list(trigrams(first_sentence)))  # [(u'ASIAN', u'EXPORTERS', u'FEAR'), (u'EXPORTERS', u'FEAR', u'DAMAGE'), (u'FEAR', u'DAMAGE', u'FROM'), ...

# Get the padded trigrams
print(list(trigrams(first_sentence, pad_left=True, pad_right=True)))  # [(None, None, u'ASIAN'), (None, u'ASIAN', u'EXPORTERS'), (u'ASIAN', u'EXPORTERS', u'FEAR'), (u'EXPO

['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.']
[('ASIAN', 'EXPORTERS'), ('EXPORTERS', 'FEAR'), ('FEAR', 'DAMAGE'), ('DAMAGE', 'FROM'), ('FROM', 'U'), ('U', '.'), ('.', 'S'), ('S', '.-'), ('.-', 'JAPAN'), ('JAPAN', 'RIFT'), ('RIFT', 'Mounting'), ('Mounting', 'trade'), ('trade', 'friction'), ('friction', 'between'), ('between', 'the'), ('the', 'U'), ('U', '.'), ('.', 'S'), ('S', '.'), ('.', 'And'), ('And', 'Japan'), ('Japan', 'has'), ('has', 'raised'), ('raised', 'fears'), ('fears', 'among'), ('among', 'many'), ('many', 'of'), ('of', 'Asia'), ('Asia', "'"), ("'", 's'), ('s', 'exporting'), ('exporting', 'nations'), ('nations', 'that'), 

In [3]:
texts = reuters.sents()

# Wonderful dataset

In [4]:
rawtext = open('wonderland.txt','r').read().split('\n')
rawtext = ' '.join(rawtext)
#rawtext = [word.strip(string.punctuation) for word in rawtext.split()]
rawtext = [word for word in rawtext.split()]
rawtext = ' '.join(rawtext)
rawtext = rawtext.replace('-', ' ')
rawtext = ' '.join(rawtext.split())

In [5]:
texts = []
for r in open('wonderland.txt','r'):
    texts.append(r)

In [6]:
#texts = rawtext

In [7]:
texts

["ALICE'S ADVENTURES IN WONDERLAND\n",
 '\n',
 'Lewis Carroll\n',
 '\n',
 'THE MILLENNIUM FULCRUM EDITION 3.0\n',
 '\n',
 'CHAPTER I. Down the Rabbit-Hole\n',
 '\n',
 'Alice was beginning to get very tired of sitting by her sister on the\n',
 'bank, and of having nothing to do: once or twice she had peeped into the\n',
 'book her sister was reading, but it had no pictures or conversations in\n',
 "it, 'and what is the use of a book,' thought Alice 'without pictures or\n",
 "conversations?'\n",
 '\n',
 'So she was considering in her own mind (as well as she could, for the\n',
 'hot day made her feel very sleepy and stupid), whether the pleasure\n',
 'of making a daisy-chain would be worth the trouble of getting up and\n',
 'picking the daisies, when suddenly a White Rabbit with pink eyes ran\n',
 'close by her.\n',
 '\n',
 'There was nothing so VERY remarkable in that; nor did Alice think it so\n',
 "VERY much out of the way to hear the Rabbit say to itself, 'Oh dear!\n",
 "Oh dear! I s

In [8]:
def build_trigram_lm(corpus):
    model = defaultdict(lambda: defaultdict(lambda: 0))

    for sentence in corpus:
        sentence = word_tokenize(sentence)
        #print(sentence)
        #print(word_tokenize(sentence))
        #print(list(trigrams(sentence, pad_right=True, pad_left=True)))
        #print(list(trigrams(word_tokenize(sentence), pad_right=True, pad_left=True)))
        for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
            model[(w1, w2)][w3] += 1

    '''
    print(model["what", "the"]["economists"])  # "economists" follows "what the" 2 times
    print(model["what", "the"]["nonexistingword"])  # 0 times
    print(model[None, None]["The"])  # 8839 sentences start with "The"
    '''
    # Let's transform the counts to probabilities
    for w1_w2 in model:
        total_count = float(sum(model[w1_w2].values()))
        for w3 in model[w1_w2]:
            model[w1_w2][w3] /= total_count
    return model


In [9]:
def sample_lm(model):
    text = [None, None]
    prob = 1.0  # <- Init probability

    sentence_finished = False

    while not sentence_finished:
        r = random.random()
        accumulator = .0

        for word in model[tuple(text[-2:])].keys():
            accumulator += model[tuple(text[-2:])][word]

            if accumulator >= r:
                prob *= model[tuple(text[-2:])][word]  # <- Update the probability with the conditional probability of the new word
                text.append(word)
                break

        if text[-2:] == [None, None]:
            sentence_finished = True

    return text, prob


In [10]:

model = build_trigram_lm(texts)
print(model["what", "the"]["economists"])  # 0.0434782608696
print(model["what", "the"]["nonexistingword"])  # 0.0
print(model[None, None]["The"])  # 0.161543241465

0
0
0.015032211882605583


In [11]:
text = sample_lm(model)
print(' '.join([t for t in text[0] if t]))

'You must be really offended . 'We wo n't interrupt again .
